## Making necessary imports

In [2]:
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import train_test_split  
from sklearn.utils import check_array
from sklearn.ensemble import RandomForestRegressor
import quandl
import requests, zipfile,io
from sklearn.metrics import r2_score

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Urls for fetching the fama french data
#### ff3 is the 3 variable model

In [3]:
url_ff3="http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip"
url_ff3_daily="http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip"

### Function to fetch the data

In [4]:
def fetchingfamadata(url,name_of_csv,no_of_skiprows,no_of_skipfooter):
    r=requests.get(url,stream=True)
    z=zipfile.ZipFile(io.BytesIO(r.content))
    x=z.extractall()
    df=pd.read_csv(name_of_csv,error_bad_lines=False,skiprows=no_of_skiprows,skipfooter=no_of_skipfooter)
    return df


### Downloading the FAMA Factor 3 model data for daily and monthly intervals

In [5]:
### Fetching the FAMA 3 Factor model data with monthly frequency
ff3=fetchingfamadata(url_ff3,'F-F_Research_Data_Factors.csv',2,1)

### Fetching the FAMA 3 Factor model data with daily frequency
ff3_daily=fetchingfamadata(url_ff3,'F-F_Research_Data_Factors_daily.csv',3,1)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """


In [6]:
ff3.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.30,-2.87,0.22
1,192608,2.64,-1.40,4.19,0.25
2,192609,0.36,-1.32,0.01,0.23
3,192610,-3.24,0.04,0.51,0.32
4,192611,2.53,-0.20,-0.35,0.31


In [7]:
ff3_daily.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,19260701,0.10,-0.24,-0.28,0.009
1,19260702,0.45,-0.32,-0.08,0.009
2,19260706,0.17,0.27,-0.35,0.009
3,19260707,0.09,-0.59,0.03,0.009
4,19260708,0.21,-0.36,0.15,0.009


### Function to remove unnecessary rows in the monthly FAMA data

In [8]:
def cleanfamamonthly(df):
    ind=(df.index[df['Unnamed: 0']==' Annual Factors: January-December ']).tolist()
    df=df.head(ind[0])
    return df

In [9]:
ff3=cleanfamamonthly(ff3)

### Extracting the year and month column to be used for merging the data later

In [10]:
def extract_month_year_day(df):
    flag=0
    if (int(df['Unnamed: 0'].max())>999999):
        flag=1
    df['Unnamed: 0']=df['Unnamed: 0'].astype('str')
    df['Year'] = df['Unnamed: 0'].apply(lambda x: x[0:4])
    df['Month'] = df['Unnamed: 0'].apply(lambda x: x[4:6]) 
    if flag==1:
        df['Day'] = df['Unnamed: 0'].apply(lambda x: x[6:])
    return df

ff3=extract_month_year_day(ff3)

ff3_daily=extract_month_year_day(ff3_daily)


In [11]:
ff3.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RF,Year,Month
0,192607,2.96,-2.30,-2.87,0.22,1926,07
1,192608,2.64,-1.40,4.19,0.25,1926,08
2,192609,0.36,-1.32,0.01,0.23,1926,09
3,192610,-3.24,0.04,0.51,0.32,1926,10
4,192611,2.53,-0.20,-0.35,0.31,1926,11


In [12]:
ff3_daily.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RF,Year,Month,Day
0,19260701,0.10,-0.24,-0.28,0.009,1926,07,01
1,19260702,0.45,-0.32,-0.08,0.009,1926,07,02
2,19260706,0.17,0.27,-0.35,0.009,1926,07,06
3,19260707,0.09,-0.59,0.03,0.009,1926,07,07
4,19260708,0.21,-0.36,0.15,0.009,1926,07,08


### Fetching the stocks data

In [13]:
api_key=input()

_PpK5yM-XqckUuWnUoHp


In [14]:
quandl.ApiConfig.api_key=api_key

### Function to calculate the MAPE value

In [15]:
def mean_absolute_percentage_error(y_test,x_predict):
    np.seterr(divide='ignore',invalid='ignore')
    y_test,x_predict=np.array(y_test),np.array(x_predict) 
    return np.mean(np.abs((y_test - x_predict)/y_test))*100


### Changing the datatypes to float so that it can be used in the model

In [17]:
def datatypechange(df,nameofcol):
    df['RF']=df['RF'].astype('float64')
    df['SMB']=df['SMB'].astype('float64')
    df['HML']=df['HML'].astype('float64')
    df['Mkt-RF']=df['Mkt-RF'].astype('float64')
    df['rt']=df[nameofcol]-df['RF']
    df=df[['rt','Mkt-RF','HML','SMB']]
        
    return df


### List of stocks 

In [18]:
stocklist=["WIKI/MMM.11", "WIKI/AXP.11", "WIKI/AAPL.11", "WIKI/BA.11", "WIKI/CAT.11", "WIKI/CVX.11", "WIKI/CSCO.11", "WIKI/KO.11", "WIKI/DWDP.11", "WIKI/DIS.11", "WIKI/XOM.11", "WIKI/GE.11", "WIKI/GS.11", "WIKI/HD.11", "WIKI/IBM.11", "WIKI/INTC.11", "WIKI/JNJ.11", "WIKI/JPM.11", "WIKI/MCD.11", "WIKI/MRK.11", "WIKI/MSFT.11", "WIKI/NKE.11", "WIKI/PFE.11", "WIKI/PG.11", "WIKI/TRV.11", "WIKI/UTX.11", "WIKI/UNH.11", "WIKI/VZ.11", "WIKI/V.11", "WIKI/WMT.11"]

### Dataframe to store the final results

In [19]:
final_result=pd.DataFrame(columns=["Stock","DataType","Model","Train_R2","Test_R2","Train_MAE","Test_MAE","Train_RMSE","Test_RMSE","Train_MAPE","Test_MAPE","Mkt_RF_coefficient","HML_coefficient","SMB_coefficient"])

### Calculating MAE

In [20]:
def mae(actual,prediction):
    return mean_absolute_error(actual,prediction)

### Calculating RMSE

In [21]:
def rmse(actual,prediction):
    return np.sqrt(mean_squared_error(actual,prediction))

#### Linear regression model and model performance metrics

In [22]:
def regression(df):   
    
    #### Splitting Data into Train and Test
    train,test = train_test_split(df, train_size = 0.7)
    
    #### Separating label from features
    y_train = train['rt']
    y_test = test['rt']
    x_train = train.loc[:, train.columns != 'rt']
    x_test = test.loc[:, test.columns != 'rt']
    
    #### Building model
    reg=linear_model.LinearRegression()
    reg.fit(x_train,y_train)
    
    #### Doing predictions on training and test data
    predicted_train=reg.predict(x_train)
    predicted_test=reg.predict(x_test)
    
    #### MAE Calculation of model
    test_mae=mae(y_test,predicted_test)
    train_mae=mae(y_train,predicted_train)
    
    #### RMSE Calculation of model
    test_rmse = rmse(y_test,predicted_test)
    train_rmse=rmse(y_train,predicted_train)
    
    #### R Squared error calculation
    test_r2=r2_score(y_test,predicted_test)
    train_r2=reg.score(x_train,y_train)
    
    
    #### Calculating MAPE
    test_mape = mean_absolute_percentage_error(y_test, predicted_test)
    train_mape = mean_absolute_percentage_error(y_train, predicted_train)
    
    ### Getting coefficients for features 
    coefficients=reg.coef_
    
    Mkt_RF_coefficient=coefficients[0]
    
    HML_coefficient=coefficients[1]
    
    SMB_coefficient=coefficients[2]
    
    return train_r2,test_r2,train_mae,test_mae,train_rmse,test_rmse,train_mape,test_mape,Mkt_RF_coefficient,HML_coefficient,SMB_coefficient
    


### Calculating Yearly monthly and daily columns for the stock data

In [23]:
def calculate_year_month_day(df):
    df.reset_index(inplace=True)
    df['Date']=df['Date'].astype('str')
    s = df['Date'].apply(lambda x: x.split('-'))
    df['Year'] = s.apply(lambda x: x[0])
    df['Month'] = s.apply(lambda x: x[1])
    df['Day'] = s.apply(lambda x: x[2])
    return df
   

### Running the main code

In [25]:

rownum=1
    
for i in stocklist:
    ### Fetching daily and monthly data
    stock_data=quandl.get([i],transform="rdiff",collapse="monthly")
    stock_data_daily=quandl.get([i],transform="rdiff",collapse="daily")
    
    ### Calculating the name of the stock data columns
    
    nameofcol=i[:-3]+' - Adj. Close'
    
    
    ### Fetching the date column for merging and calculating the day month and year for merging with fama data
    stock_data=calculate_year_month_day(stock_data)
    stock_data_daily=calculate_year_month_day(stock_data_daily)
    
    ### Bringing the stock data to the scale of the FAMA Data
    stock_data[nameofcol]=stock_data[nameofcol]*100
    
    
    ### Bringing the stock data to the scale of the FAMA Data
    stock_data_daily[nameofcol]=stock_data_daily[nameofcol]*100
   
    ### This is a counter to add result to the dataframe
    if rownum>1:
        rownum+=1
    
    
    ### merging the monthly stock data with the FAMA 3 factor monthly data
    
    result_ff3=stock_data.merge(ff3,how="inner",left_on=["Year","Month"],right_on=["Year","Month"])
    result_ff3=datatypechange(result_ff3,nameofcol)
    
    ### Creating the monthly data model and adding the inputs to the result dataset
    train_r2,test_r2,train_mae,test_mae,train_rmse,test_rmse,train_mape,test_mape,Mkt_RF_coefficient,HML_coefficient,SMB_coefficient=regression(result_ff3)
    final_result.loc[rownum]=[i[5:-3],"Monthly","3-Factor",train_r2,test_r2,train_mae,test_mae,train_rmse,test_rmse,train_mape,test_mape,Mkt_RF_coefficient,HML_coefficient,SMB_coefficient]
    
    
    
    
    
    ### merging the monthly stock data with the FAMA 3 factor daily data
    
    result_ff3_daily=stock_data_daily.merge(ff3_daily,how="inner",left_on=["Year","Month","Day"],right_on=["Year","Month","Day"])
    result_ff3_daily=datatypechange(result_ff3_daily,nameofcol)
    
    ### Creating the daily data model and adding the inputs to the result dataset
    
    train_r2,test_r2,train_mae,test_mae,train_rmse,test_rmse,train_mape,test_mape,Mkt_RF_coefficient,HML_coefficient,SMB_coefficient=regression(result_ff3_daily)
    rownum+=1
    final_result.loc[rownum]=[i[5:-3],"Daily","3-Factor",train_r2,test_r2,train_mae,test_mae,train_rmse,test_rmse,train_mape,test_mape,Mkt_RF_coefficient,HML_coefficient,SMB_coefficient]
    

In [26]:
print(final_result)

   Stock DataType     Model  Train_R2   Test_R2     Train_MAE  Test_MAE  \
1    MMM  Monthly  3-Factor  0.353970  0.429729  3.534166e+00  3.469760   
2    MMM    Daily  3-Factor  0.391461  0.440039  7.783516e-01  0.775466   
3    AXP  Monthly  3-Factor  0.432492  0.590767  4.472161e+00  4.495657   
4    AXP    Daily  3-Factor  0.491321  0.441542  1.117161e+00  1.122284   
5   AAPL  Monthly  3-Factor  0.259721  0.265044  8.745882e+00  8.492235   
6   AAPL    Daily  3-Factor  0.229497  0.281071  1.708380e+00  1.685444   
7     BA  Monthly  3-Factor  0.324869  0.190645  6.030290e+00  5.996897   
8     BA    Daily  3-Factor  0.264786  0.240023  1.226242e+00  1.245878   
9    CAT  Monthly  3-Factor  0.355174  0.422703  5.213194e+00  4.697579   
10   CAT    Daily  3-Factor  0.315387  0.321388  1.065689e+00  1.088051   
11   CVX  Monthly  3-Factor  0.259960  0.218893  4.205083e+00  4.347583   
12   CVX    Daily  3-Factor  0.287180  0.359693  9.417233e-01  0.971498   
13  CSCO  Monthly  3-Fact

### Outputting the result to a csv

In [20]:
final_result.to_csv('FAMAResult.csv',index=False)